In [30]:
import numpy as np
import tensorflow as tf
import random

In [43]:
## Hyper-Parameters
n = 1000 # number of samples
d = 10  # input dimension
k = 5   # hidden layer size
kappa = 2 
T = 10 # num of epochs
lr = 0.02
batch_size = 1

In [32]:
## Activation Function
def phi(h):
    return h**2 if h > 0 else 0
phi = np.vectorize(phi)

In [49]:
## Data and Params Generation

gauss_mat = np.random.normal(0.0, 0.1 , (d,k))
U, V = np.linalg.qr(gauss_mat)

diag = []
v_gt = []
v_choice = [1,-1]
for iter in range(k):
    diag.append(1+1.*iter*(kappa-1)/(k-1))
    v_gt.append(random.choice(v_choice))
    
Sigma = np.diag(diag)
W_gt = np.dot(np.dot(U, Sigma), np.transpose(V))
v_gt = np.asarray(v_gt)

train_x = []
train_y = []
for iter in range(n):
    train_x.append(np.random.normal(0.0,1.,d))
    train_y.append(np.dot(phi(np.dot(train_x, W_gt)),v_gt))
train_x = np.asarray(train_x)
# train_y = np.asarray(train_y)

In [34]:
### Tensor Initialization


In [37]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    """
    h    = tf.square(tf.nn.relu((tf.matmul(X, w_1))))
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

In [50]:
X = tf.placeholder("float", shape=[None, d])
y = tf.placeholder("float", shape=[None, 1])

# Weight initializations
w_1 = init_weights((d, k))
w_2 = init_weights((k, 1))

# Forward propagation
yhat  = forwardprop(X, w_1, w_2)

# Backward propagation
cost = tf.losses.mean_squared_error(y, yhat)
updates = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# Run SGD
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(T):
    # Train with each example
    i = 0
    for iter in range(int(n/batch_size)):
        sess.run(updates, feed_dict={X: train_x[i: i + batch_size], y: train_y[i: i + batch_size]})
        i = (i + batch_size)%n

ValueError: Cannot feed value of shape (1, 2) for Tensor 'Placeholder_23:0', which has shape '(?, 1)'